In [76]:
# # Installations for your venv
# %pip install numpy
# %pip install pandas
   # to split strings using regex


# Imports and Variables

In [77]:
import os
import sys
import re
import pandas as pd
import numpy as np
import argparse

Parse command line

## Parse file - Using DataFrames

In [78]:
def process_slots(data: str, columns: list, event_type: str):
    """
    Processes a string of slot data and converts it into a pandas DataFrame.
    Args:
        data (str): A string containing slot data, with each slot separated by a newline.
        columns (list): A list of column names for the DataFrame.
        event_type (str): A character indicating either games 'G' or practices 'P'
    Returns:
        pd.DataFrame: A DataFrame containing the processed slot data.
    """
    slots = []
    indices = []
    for slot in data.split('\n'):
        slot = slot.replace(' ','')
        if len(slot) > 0:
            print(slot)
            day, time, max, min = slot.strip().split(',')
            # if not properties == [''] :
            slots.append({'Day': day,
                          'Start': time,
                          'Max': max,
                          'Min': min})
            indices.append(day + time)
    df = pd.DataFrame(slots, columns=columns, index=indices)
    df['Invalid_Assign'] = np.empty((len(df), 0)).tolist()
    df['Type'] = event_type
    return df


**Example of the Slots table**
|  Index  | Day     | Time    | Gamemin | Gamemax | Count   | Invalid_Assign                      | Type   |
|  -----  | ------- | ------- | ------- | ------- | ------- | ----------------------------------- | ------ |
| MO10:00 | MO      | 10:00   | 3       | 4       | 3       | [CMSAU17T2PRC01, CMSAU13T3DIV02]    | G      |

Example for verification:
1. Game max hard constraint

     ```slotindex[Count] > slotindex[Gamemax]```
2. Game min soft constraint

     ```slotindex[Count] > slotindex[Gamemin]```
3. Incompatible, Divs, Tiers hard constraints

     if most recent assignment is x, check if x is in slotindex[Invalid_Assign] ?

The following columns are filled during parsing and do not change:
- Index
- Day
- Time 
- Gamemin
- Gamemax

The following would need to be updated as assignments are made:
- Count: Increment count of appropriate slot for each assignment 
- Incompatible: Add incompatible pair 'partner' of assigned event (See intended lookup in #3).
     - We could also create a list of all events with a given 'tier' by filtering the games/practices dataframes.

In [79]:
def get_index(event: str):
    """
    Convert a string to desired index in DataFrame
    """
    return event.replace(' ', '')

In [80]:
def process_games_practices(data, columns, event_type):
    """
    Processes game and practice data and converts it into a pandas DataFrame.
    Args:
        data (str): The raw data as a string, where each line represents a game or practice entry.
        columns (list): A list of column names for the resulting DataFrame.
    Returns:
        pd.DataFrame: A DataFrame containing the processed game and practice data.
    """
    
    items = []
    indices = []
    strings = data.split('\n')
    for item in strings:
        index = get_index(item)
        # print(index)
        if len(index) > 0: # If not empty
            list_attributes = re.split(r'\s+', item.strip())
            if event_type == 'P' and len(list_attributes) == 6: # Normal practice
                print(item)
                league, tier, _, divnum, ptype, pnum = list_attributes
            elif event_type == 'P' and len(list_attributes) == 4: # Practice used by all Divs
                print(item)
                league, tier, ptype, pnum = list_attributes
                divt = 'DIV'
                divnum = ''
            elif event_type == 'G' and len(list_attributes) == 4: # Game
                print(item)
                league, tier, _, divnum = list_attributes
                ptype = ''
                pnum = ''
            else: # Something is wrong
                raise ValueError(f"Invalid data format for event type {event_type}: {item}")
            # Dictionary of this item
            items.append({'League': league,
                          'Tier': tier,
                          'Div': divnum,
                          'Practice_Type': ptype,
                          'Num': pnum})
            indices.append(index)
    # DataFrame with items
    df = pd.DataFrame(items, index = indices)
    df['Type'] = event_type
    return df


**Example of the Games Lookup Table**
|  Index           | League     | Tier    | Div     | Unwanted | Pref_slot   | Pref_value | Incompatible     | Type | Assigned |
|  --------------  | ---------- | ------- | ------- | -------- | ----------- | ---------- | ---------------- | ---- | -------- |
|  CMSAU13T3DIV02  | CMSA       | U13T3   | DIV02   | TU9:30   | MO8:00      | 10         | [CMSAU19T3DIV01] | G    |          |


**Example of the Practices Lookup Table**
|  Index                | League     | Tier    | Div     | Type     | Num      | Unwanted | Pref_slot   | Pref_value | Incompatible     | Type | Assigned |
|  -------------------  | ---------- | ------- | ------- | -------- | -------- | -------- | ----------- | ---------- | ---------------- | ---- | -------- |
|  CMSAU13T3DIV02OPN03  | CMSA       | U13T3   | DIV02   | OPN      | 03       | TU10:00  | TU17:00     | 10         | [CMSAU13T3DIV02] | P    |

**Inner Join creating Events Lookup Table**
|  Index                | League     | Tier    | Div     | Type     | Num      | Unwanted | Pref_slot   | Pref_value | Incompatible     | Type | Assigned |
|  -------------------  | ---------- | ------- | ------- | -------- | -------- | -------- | ----------- | ---------- | ---------------- | ---- | -------- |
|  CMSAU13T3DIV02OPN03  | CMSA       | U13T3   | DIV02   | OPN      | 03       | TU10:00  | TU17:00     | 10         | [CMSAU13T3DIV02] | P    | MO10:00
|  CMSAU13T3DIV02       | CMSA       | U13T3   | DIV02   |          |          | TU9:30   | MO8:00      | 5          | [CMSAU19T3DIV01] | G    |

Example Verification method:
1. Unwanted hard constraint

     ```if events['unwanted'] == events['Assigned']```
2. Preference soft constraint

     ```if assigned /= events['Pref_slot']``` then use ```events['Pref_value']``` to calculate penalty



-- Ideas:
To calculate the eval function, we can just inner join the Game_slots with the games dataframe and practice slots with the practices dataframe and vectorize values


List of strings of games and practices

In [81]:
# Strings Example
# games2 = split_data[3].split('\n')

# practice2 = split_data[4].split('\n')

def get_prop(event: str, feature):
    """
    Extracts a specific property from an event string based on the given feature.
    Parameters:
    event (str): The event string containing various properties separated by spaces or 'DIV'.
    feature (str): The feature to extract from the event string. 
                   Valid options are "League", "Tier", "Div", "Type", and "Num".
    Returns:
    str: The extracted property based on the specified feature.
    Raises:
    ValueError: If the feature is invalid or if the event string does not contain the required number of properties.
    """
    properties = event.strip().split(r'\s+| DIV')
    if feature == "League":
        return properties[0]
    elif feature == "Tier":
        return properties[1]
    elif feature == "Div":
        return properties[2]
    elif feature == "Type":
        if len(properties) >3:
            return properties[3]
        else: raise ValueError("Called Type on game or Type property is missing in the event data")
    elif feature == "Num":
        if len(properties) > 4:
            return properties[4]
        else: raise ValueError("Called Num on game or Type property is missing in the event data")
    else:
        raise ValueError("Invalid feature")

In [82]:
class Env:
    def __init__(self):
        """
        Initializes the class instance by performing the following steps:
        1. Prompts the user to input a file name and a series of integers representing weights and penalties.
        2. Checks if the specified file exists. If it does, reads and processes the file content.
        3. Validates that the input integers are positive.
        4. Processes game slots and practice slots from the file data.
        5. Processes games and practices into lookup tables.
        6. Combines games and practices into a single DataFrame.
        7. Initializes columns for unwanted slots, incompatible events, paired events, and partial assignments.
        8. Adds unwanted slots to the 'Unwanted' column for each event.
        9. Adds slot preferences to the 'Pref' and 'Pref_value' columns for each event.
        10. Adds paired events to the 'Pair_with' column for each event.
        11. Processes partial assignments and updates the 'Part_assign' column for each event.
        Attributes:
            w_minfilled (int): Weight for minimum filled slots.
            w_pref (int): Weight for preferences.
            w_pair (int): Weight for pairs.
            w_secdiff (int): Weight for section differences.
            pen_gamemin (int): Penalty for minimum games.
            pen_pracmin (int): Penalty for minimum practices.
            pen_notpaired (int): Penalty for not paired events.
            pen_section (int): Penalty for section issues.
            game_slots (DataFrame): Processed game slots.
            practice_slots (DataFrame): Processed practice slots.
            events (DataFrame): Combined DataFrame of games and practices with additional columns for unwanted slots, preferences, pairs, and partial assignments.
        """
        
        
        file_name = input('File')
        integers = input('Weights and Penalties:').split(' ')
        integers = list(map(int, integers))
        print(f'File chosen = {file_name}')
        print(f'Integer inputs = {integers}')
        # sys.argv = [input1].append(input2)
    
        # Command line parser command line input is always file name and integers.

        # check if file exists
        if os.path.isfile(file_name):
            with open(file_name, "r") as inputfile:   # opens file     
                data = inputfile.read()          # starts reading file
                # splits the file based on the key words
                
                split_data = re.split(r"Name:|Game slots:|Practice slots:|Games:|Practices:|Not compatible:|Unwanted:|Preferences:|Pair:|Partial assignments:", data, flags=re.IGNORECASE)
                print(split_data)
        else: 
            print("Unable to open file. Please try again.")
            
        # check if integers are positive
        if any(i < 0 for i in integers):
            print("Please enter 8 positive integers.")
        else:
            self.w_minfilled, self.w_pref, self.w_pair, self.w_secdiff, self.pen_gamemin, self.pen_pracmin, self.pen_notpaired, self.pen_section = integers
            
        # Process game slots
        self.game_slots = process_slots(split_data[2], ['Day', 'Start', 'Max', 'Min', 'Invalid_Assign'], 'G')
            
        # Process practice slots
        self.practice_slots = process_slots(split_data[3], ['Day', 'Start', 'Max', 'Min', 'Invalid_Assign'], 'P')
        
        
        # Process games into a lookup table
        games = process_games_practices(split_data[4], ['League', 'Tier', 'Div'], 'G')

        # Process practices into a lookup table
        practices = process_games_practices(split_data[5], ['League', 'Tier', 'Div', 'Practice_Type', 'Num'], 'P')

        # Combine Practices and Games
        events = pd.concat([games, practices], axis=0)

        # Prepare columns of empty lists
        events['Unwanted'] = np.empty((len(events), 0)).tolist()
        events['Incompatible'] = np.empty((len(events), 0)).tolist()
        events['Pair_with'] = np.empty((len(events), 0)).tolist()
        # events['Part_assign'] = np.empty((len(events), 0)).tolist()    
        
        # Add all unwanted slots to the list in 'Unwanted' column for each event mentioned
        for unwanted in split_data[7].split('\n'):
            unwanted = unwanted.replace(' ','')
            if len(unwanted)>0:
                print(unwanted)
                event, day, time = unwanted.strip().split(',')
                if not event in events.index:
                    print(f'Entry error: {event} is not in table')
                else:
                    event = get_index(event)
                    events.at[event, 'Unwanted'].append(day + time)
                    
        # Add all slot preferences to the preference and preference value
        # columns for each event mentioned
        for pref in split_data[8].split('\n'):
            pref = pref.replace(' ','')
            if len(pref) > 0:
                print(pref)
                day, time, index, value = pref.strip().split(',')
                if not index in events.index:
                    print(f'Entry error: {index} is not in table')
                else:
                    events.at[index, 'Pref'] = day + time
                    events.at[index, 'Pref_value'] = value
        events['Pref'] = events['Pref'].fillna('')
        events['Pref_value'] = events['Pref_value'].fillna(0)

        for pair in split_data[9].split('\n'):
            string = pair.replace(' ', '')
            if len(string) > 0: # If not empty
                print(string)
                event1, event2 = string.strip().split(',')
                if event1 in events.index and event2 in events.index:
                    events.at[event1, 'Pair_with'].append(event2)
                    events.at[event2, 'Pair_with'].append(event1)
                    
                    available = events.index.tolist()
        print(available)
        # Remove events from this list as they are assigned to keep track

        # Partial assignments used when f_trans selects branchs, before random choices
        partial_assignments = []
        for assign in split_data[10].split('\n'):
            string = assign.replace(' ', '')
            if len(string) > 0: # If not empty
                team, day, time = string.strip().split(',')
                partial_assignments.append((team, day+time))
                events.at[team, 'Part_assign'] = day+time
        print(partial_assignments)
        events['Part_assign'] = events['Part_assign'].fillna('')

        # # Strings Example
        # games2 = split_data[3].split('\n')

        # practice2 = split_data[4].split('\n')
        
        self.events = events
        
    def get_events(self):
        """
        Returns the DataFrame containing the processed game and practice data.
        Returns:
            pd.DataFrame: A DataFrame containing the processed game and practice data.
        """
        return self.events
    
    def get_game_slots(self):
        """
        Returns the DataFrame containing the processed game slots.
        Returns:
            pd.DataFrame: A DataFrame containing the processed game slots.
        """
        return self.game_slots
    
    def get_practice_slots(self):
        """
        Returns the DataFrame containing the processed practice slots.
        Returns:
            pd.DataFrame: A DataFrame containing the processed practice slots.
        """
        return self.practice_slots

In [83]:
env = Env()
events = env.get_events()
game_slots = env.get_game_slots()
practice_slots = env.get_practice_slots()

File chosen = ShortExample.txt
Integer inputs = [1, 0, 1, 0, 10, 10, 10, 10]
['', ' \nShortExample \n\n', ' \nMO, 8:00, 3, 2 \nMO, 9:00,3,2 \nTU, 9:30, 2, 1 \n\n', ' \nMO, 8:00, 4, 2 \nTU, 10:00,2,1 \nFR, 10:00, 2, 1 \n\n', ' \nCMSA U13T3 DIV  01 \nCMSA U13T3 DIV 02 \nCUSA O18 DIV 01 \nCMSA U17T1 DIV 01 \n\n', ' \nCMSA U13T3 DIV 01 PRC 01 \nCMSA U13T3 DIV 02 OPN 02 \nCUSA O18 DIV 01 PRC 01 \nCMSA U17T1 PRC 01 \n\n', ' \nCMSA U13T3 DIV 01 PRC 01, CMSA U13T3 DIV 02 OPN 02 \nCMSA U17T1 DIV 01, CMSA U13T3 DIV 01 \nCMSA  U17T1 DIV 01, CMSA U13T3 DIV 02 \nCMSA U17T1 PRC 01, CMSA U13T3 DIV 02 \nCMSA U13T3 DIV 01, CMSA U17T1 PRC 01 \n\n', ' \nCMSA U13T3 DIV 01, MO, 8:00 \n\n', ' \nTU, 9:30, CMSA U13T3 DIV 01, 10 \nMO, 8:00, CMSA U13T3 DIV   01 PRC 01, 3 \nTU, 9:30, CMSA U13T3 DIV 02, 10 \nTU, 10:00, CMSA U13T3 DIV 01 OPN 02, 5 \nMO, 8:00, CMSA U13T3 DIV 02 OPN 02, 1 \nMO, 10:00, CMSA U13T3 DIV 02 OPN 02, 7 \n\n', ' \nCUSA O18 DIV 01, CMSA U17T1 DIV 01 \n\n', ' \nCUSA O18 DIV 01, MO, 8:00 \nCUS

Tracker for games and practices

In [84]:
events

,League,Tier,Div,Practice_Type,Num,Type,Unwanted,Incompatible,Pair_with,Pref,Pref_value,Part_assign
CMSAU13T3DIV01,CMSA,U13T3,01,,,G,[MO8:00],[],[],TU9:30,10,
CMSAU13T3DIV02,CMSA,U13T3,02,,,G,[],[],[],TU9:30,10,
CUSAO18DIV01,CUSA,O18,01,,,G,[],[],[CMSAU17T1DIV01],,0,MO8:00
CMSAU17T1DIV01,CMSA,U17T1,01,,,G,[],[],[CUSAO18DIV01],,0,
CMSAU13T3DIV01PRC01,CMSA,U13T3,01,PRC,01,P,[],[],[],MO8:00,3,
CMSAU13T3DIV02OPN02,CMSA,U13T3,02,OPN,02,P,[],[],[],MO10:00,7,
CUSAO18DIV01PRC01,CUSA,O18,01,PRC,01,P,[],[],[],,0,FR10:00
CMSAU17T1PRC01,CMSA,U17T1,,PRC,01,P,[],[],[],,0,


In [85]:
# Class of schedule
class Schedule:
    def __init__(self):
        self.eval = None
        self.event_list = events.index.tolist()
        print(self.event_list)
        self.assignments = events['Part_assign']
        # self.assignments = pd.DataFrame(self.event_list, columns = ['Event'])
        
    def get_Starting(self):
        return self.assignments
        
    def set_Eval(self):
        total_df = events.copy()
        # total_df.reset_index(total_df, inplace = True, 
        total_df['Assigned'] = self.assignments
        
        df = pd.merge(total_df, game_slots, how = 'left', left_on = 'Assigned', right_index = True)
        df = pd.merge(df, practice_slots, how = 'left', left_on = 'Assigned', right_index = True)    
        
        # self.eval = pd.sum(df['Eval'])
        
    
    def assign(self, slots):
        self.assignments['Slot'] = slots
        self.set_Eval(self)
        return self.eval
    
    # def toList(self):


In [86]:
sched = Schedule()
print(sched.get_Starting())
sched.set_Eval()

['CMSAU13T3DIV01', 'CMSAU13T3DIV02', 'CUSAO18DIV01', 'CMSAU17T1DIV01', 'CMSAU13T3DIV01PRC01', 'CMSAU13T3DIV02OPN02', 'CUSAO18DIV01PRC01', 'CMSAU17T1PRC01']
CMSAU13T3DIV01                
CMSAU13T3DIV02                
CUSAO18DIV01            MO8:00
CMSAU17T1DIV01                
CMSAU13T3DIV01PRC01           
CMSAU13T3DIV02OPN02           
CUSAO18DIV01PRC01      FR10:00
CMSAU17T1PRC01                
Name: Part_assign, dtype: object
